
# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn')


/Users/floydluo/Library/CloudStorage/OneDrive-DRFIRST.COM,INC/Documents - DrFirst-JHU Collaboration Center/DrFirst-Project/2024-DrFirst-v4-SPACE/_DrFirst-AI-EgmPred-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseFeat': '../_Data/3-Data_CaseFeat',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_CaseSet': '../_Data/4-Data_CaseSet',
 'DATA_EXTERNAL': '../external',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': '_data',
 'MODEL_TASK': '../_Model',
 'TaskName': 'EduRxPred',
 'WORKSPACE_PATH': '/Users/floydluo/Library/CloudStorage/OneDrive-DRFIRST.COM,INC/Documents '
                   '- DrFirst-JHU Collaboration '
                   'Center/DrFirst-Project/2024-DrFirst-v4-SPACE/_DrFirst-AI-EgmPred-WorkSpace'}


# Trigger Events

In [2]:
import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args
from recfldtkn.loadtools import convert_variables_to_pystirng, load_module_variables

cohort_config = load_cohort_args(recfldtkn_config_path, SPACE)
print(cohort_config)

{'CohortInfo': {'RawData2023Nov_Trulicity': {'cohort_label': 1, 'cohort_name': 'RawData2023Nov_Trulicity', 'FolderPath': '../_Data/0-Data_Raw/2023_11_30_30days_Trulicity'}}, 'RawRootID': 'patient_id_encoded', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_Sequence': ['P', 'PInv', 'Rx', 'EgmAuthen', 'EgmCallPharm', 'EgmClick', 'EgmCopay', 'EgmEdu', 'EgmRmd'], 'RecName_to_PrtRecName': {'P': 'None', 'PInv': 'P', 'Rx': 'PInv', 'EgmAuthen': 'PInv', 'EgmCallPharm': 'PInv', 'EgmClick': 'PInv', 'EgmCopay': 'Rx', 'EgmEdu': 'Rx', 'EgmRmd': 'Rx'}, 'RecName_to_RFT_GROUP_SIZE': {'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'Default': 10000}, 'RecName_to_RFT_usebucket': {'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_path': '../pipeline/config_r

# TriggerCaseMethod: TrulicityRx

In [3]:
######################################
TriggerCaseMethod = 'AnyInv'
######################################

#################
cohort_label_list = [1]
#################


##############
TriggerRecName = 'PInv' # 'PInv'
case_id_columns = ["PID", "ObsDT", 'PInvID']
special_columns = ['PID', 'DT', 'PInvID', 'invitation_type', 'invitation_state', 
                   ]
##############

In [4]:

import inspect

def convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, base_config):
    ds_case = ds_rec.select_columns(special_columns)
    df_case = ds_case.to_pandas()
    
    # ------------------------------ Filter Cases ------------------------------
    # df_case_filter = df_case_raw[df_case_raw['invitation_state'] == 'Delivered'].reset_index(drop=True)
    # --------------------------------------------------------------------------

    # ------------------------------ Determine the ObsDT ------------------------------
    df_case['ObsDT'] = df_case['DT'] # [df_case_raw['drug_name'] == 'Trulicity']
    # --------------------------------------------------------------------------

    # ------------------------------- Update Column Sequence ------------------------
    columns = df_case.columns 
    columns = case_id_columns + [col for col in columns if col not in case_id_columns]
    df_case = df_case[columns].reset_index(drop=True)

    return df_case

convert_TriggerEvent_to_Caseset.fn_string = inspect.getsource(convert_TriggerEvent_to_Caseset)

In [5]:
prefix = ['import pandas as pd', 'import numpy as np']
string_variables = [TriggerRecName]
iterative_variables = [case_id_columns, special_columns]
fn_variables = [convert_TriggerEvent_to_Caseset]
pycode = convert_variables_to_pystirng(string_variables = string_variables, 
                                       iterative_variables = iterative_variables, 
                                       fn_variables = fn_variables, prefix = prefix)
pypath = os.path.join(cohort_config['trigger_pyfolder'], f'{TriggerCaseMethod}.py')
print(pypath)
if not os.path.exists(os.path.dirname(pypath)): os.makedirs(os.path.dirname(pypath))
with open(pypath, 'w') as file: file.write(pycode)

../pipeline/fn_trigger/AnyInv.py


In [6]:
pypath = os.path.join(cohort_config['trigger_pyfolder'], f'{TriggerCaseMethod}.py')
module = load_module_variables(pypath)

######################
TriggerRecName = module.TriggerRecName
case_id_columns = module.case_id_columns
special_columns = module.special_columns
convert_TriggerEvent_to_Caseset = module.convert_TriggerEvent_to_Caseset
######################

In [7]:
############################### ds_rec to df_case ###############################
ds_rec, _ = load_ds_rec_and_info(TriggerRecName, cohort_config)
print(ds_rec)
df_case = convert_TriggerEvent_to_Caseset(ds_rec, case_id_columns, special_columns, cohort_config)
df_case

Dataset({
    features: ['PID', 'PInvID', 'invitation_id_encoded', 'patient_invitation_id_encoded', 'DT', 'invitation_date', 'invitation_code', 'invitation_type', 'invitation_state', 'workflow_step', 'PInv-InfoTkn_wgt', 'PInv-InfoTkn_tknidx'],
    num_rows: 526327
})


,PID,ObsDT,PInvID,DT,invitation_type,invitation_state
0,1000001,2023-04-07 18:44:16.912,1000001-000,2023-04-07 18:44:16.912,PrescribingEvent,Delivered
1,1000001,2023-04-08 19:03:59.664,1000001-001,2023-04-08 19:03:59.664,Followup,SMSBlocked
2,1000001,2023-05-08 16:06:45.254,1000001-002,2023-05-08 16:06:45.254,PrescribingEvent,Ignored
3,1000001,2023-05-10 16:45:29.112,1000001-003,2023-05-10 16:45:29.112,PrescribingEvent,Ignored
4,1000001,2023-06-07 11:25:15.721,1000001-004,2023-06-07 11:25:15.721,PrescribingEvent,Ignored
...,...,...,...,...,...,...
526322,1046209,2023-07-19 16:24:52.119,1046209-003,2023-07-19 16:24:52.119,Followup,Delivered
526323,1046209,2023-07-20 15:05:52.683,1046209-004,2023-07-20 15:05:52.683,PrescribingEvent,Delivered
526324,1046209,2023-09-07 18:10:13.007,1046209-005,2023-09-07 18:10:13.007,PrescribingEvent,Delivered
526325,1046209,2023-09-08 18:45:17.580,1046209-006,2023-09-08 18:45:17.580,Followup,SMSBlocked


In [8]:
# df_case['invitation_state'].value_counts()

In [9]:
CaseFolder = os.path.join(SPACE['DATA_CaseSet'])

if not os.path.exists(CaseFolder): os.makedirs(CaseFolder)
path = os.path.join(CaseFolder, f'{TriggerCaseMethod}.p')
df_case.to_pickle(path)
print(path)

../_Data/4-Data_CaseSet/AnyInv.p
